In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e6:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73290%2F8710574%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240618%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240618T140804Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D686754c232ebcc072fb23c251aa69c75e5177a54d3dc6ae4cc0c713988b30ffb63832cce7f1a0912328fabe604fe325058c4f77661ee6210b859dd67ce1dd67afe96b404992068a909a0f6aadb5821dc9c4aa557ae2facf41907ff333de974135c77b3f41f5433a60504f1faa1ef3dae8cec0d20bd26465c93b23a96afdd5c92d832df46e599918452f4f3567780a8e84fa6f5b68900d1f117354ab088c5d06f00c29c3982486399a3a31e6eea8cc3818fd601affeb837e07aabcaf90f52588f7e9028f19834c35fff53ec12655f78bf16690c392555a2a71986e1a26afe26e213adfdd08ac32528d31b43814b605d6d2bdf55259d2c97c06c92c293fa32b698'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 3217279 bytes downloaded
Downloaded and uncompressed: playground-series-s4e6
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv
/kaggle/input/playground-series-s4e6/sample_submission.csv


In [3]:
train_df=pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
test_df=pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
train_df

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


In [4]:
test_df

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,1,19,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,1,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [5]:
train_df["Target"] = train_df["Target"].replace({"Graduate":0 ,"Dropout":1,"Enrolled":2})

In [ ]:
X = train_df.drop(['Target'], axis=1)
y = train_df[['Target']]
X

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler

# Sample data for demonstration purposes
data_train = {
    'feature1': np.random.rand(10),
    'feature2': np.random.rand(10),
    'feature3': np.random.rand(10)
}

data_test = {
    'feature1': np.random.rand(5),
    'feature2': np.random.rand(5),
    'feature3': np.random.rand(5)
}

# Create DataFrames
X = pd.DataFrame(data_train)
test_df = pd.DataFrame(data_test)

# Fit the RobustScaler on the training data
scaler = RobustScaler()
scaler.fit(X)

# Transform the training data
X_scaled = scaler.transform(X)

# Transform the test data
test_df_scaled = scaler.transform(test_df)

# Convert the scaled arrays back to DataFrames for better readability
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
test_df_scaled_df = pd.DataFrame(test_df_scaled, columns=test_df.columns)

print("Scaled Training Data:")
print(X_scaled_df.head())

print("Scaled Test Data:")
print(test_df_scaled_df.head())


Scaled Training Data:
   feature1  feature2  feature3
0 -2.048466  0.011097  0.825406
1  0.427761 -0.011097  0.131213
2 -0.653967 -0.842165 -0.793752
3  0.545130  0.020386  0.524854
4  0.300496  0.740270 -0.125478
Scaled Test Data:
   feature1  feature2  feature3
0 -1.623554 -0.819844 -0.955070
1  0.727860 -0.171357  0.494719
2  0.530252 -0.202931  0.976796
3  0.687221 -1.267173 -1.307355
4 -0.852088 -1.071894 -0.302048


In [10]:
test_df.shape

(5, 3)

In [11]:
print("\nScaled Test Data:")
print(test_df)


Scaled Test Data:
   feature1  feature2  feature3
0  0.196953  0.338891  0.194153
1  0.884358  0.605740  0.694351
2  0.826589  0.592747  0.860674
3  0.872477  0.154817  0.072610
4  0.422481  0.235173  0.419455


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

NameError: name 'y' is not defined

In [13]:
!pip install catboost optuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.1 MB/s eta 0:00:00


In [14]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from catboost import CatBoostClassifier
# import optuna

# # Assuming X and y are your features and target
# # For illustration, let's create some dummy data
# # X, y = some_data_loading_function()

# # Split the data (assuming X and y are already defined)
# # Example: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define the objective function
# def objective(trial):
#     param = {
#         'iterations': trial.suggest_int('iterations', 100, 1000),
#         'depth': trial.suggest_int('depth', 4, 10),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10),
#         'border_count': trial.suggest_int('border_count', 32, 255),
#         'random_strength': trial.suggest_loguniform('random_strength', 1e-2, 10),
#         'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#         'od_type': 'Iter',
#         'od_wait': trial.suggest_int('od_wait', 10, 50)
#     }

#     model = CatBoostClassifier(**param, verbose=0)
#     model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)

#     preds = model.predict(X_test)
#     accuracy = accuracy_score(y_test, preds)
#     return accuracy

# # Create a study and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get the best parameters
# print('Best trial:')
# trial = study.best_trial
# print('  Value: {}'.format(trial.value))
# print('  Params: ')
# for key, value in trial.params.items():
#     print('    {}: {}'.format(key, value))

# # Train the final model using the best parameters
# best_params = trial.params
# best_model = CatBoostClassifier(**best_params)
# best_model.fit(X_train, y_train)

# # Evaluate the model on the test data
# preds = best_model.predict(X_test)
# final_accuracy = accuracy_score(y_test, preds)
# print('Final Accuracy on test data: {}'.format(final_accuracy))


In [16]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler

# Sample data for demonstration purposes
data = {
    'feature1': np.random.rand(100),
    'feature2': np.random.rand(100),
    'feature3': np.random.rand(100),
    'target': np.random.randint(0, 2, 100)
}

# Create DataFrame
df = pd.DataFrame(data)

# Define features and target
X = df.drop(columns='target')
y = df['target']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply RobustScaler to the data
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled arrays back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Initialize the CatBoostClassifier
model = CatBoostClassifier(
    iterations=464,
    depth=6,
    learning_rate=0.09895241567562836,
    l2_leaf_reg=9.985962934032957,
    border_count=37,
    random_strength=0.12603980788866026,
    bagging_temperature=0.05779942338447749,
    od_wait=42,
    eval_metric='Accuracy',
    verbose=100
)

# Train the model
model.fit(X_train_scaled, y_train, eval_set=(X_test_scaled, y_test), early_stopping_rounds=100)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")


0:	learn: 0.8250000	test: 0.5500000	best: 0.5500000 (0)	total: 53.1ms	remaining: 24.6s
100:	learn: 1.0000000	test: 0.6000000	best: 0.7000000 (8)	total: 250ms	remaining: 900ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7
bestIteration = 8

Shrink model to first 9 iterations.
Test Accuracy: 0.7


In [17]:
pred = model.predict(test_df)
np.unique(pred)

array([0, 1])

In [18]:
(pred).shape

(5,)

In [19]:
class_mapping = {0: 'Graduate', 1: 'Dropout', 2: 'Enrolled',}

In [20]:
pred_mapped = np.vectorize(class_mapping.get)(pred)
(pred_mapped)

array(['Dropout', 'Dropout', 'Dropout', 'Graduate', 'Graduate'],
      dtype='<U8')

In [21]:
test_df1=pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')
test_df1

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,1,19,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,1,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [22]:
pred_mapped.shape

(5,)

In [23]:
test_df1['id'].shape

(51012,)

In [25]:
# Ensure predictions are a 1-dimensional array
if pred_mapped.ndim > 1:
    pred_mapped = pred_mapped.flatten()

import pandas as pd

# Assuming test_df1 is your DataFrame containing the 'id' column
# and pred_mapped contains your predictions.

# Example of creating test_df1 and pred_mapped for demonstration
# Replace these with your actual data
test_df1 = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'feature1': [0.1, 0.2, 0.3, 0.4, 0.5],
    'feature2': [1.1, 1.2, 1.3, 1.4, 1.5]
})

pred_mapped = [0, 1, 0, 1, 0]  # Replace with your actual predictions

# Create submission DataFrame
submission = pd.DataFrame({'id': test_df1['id'], 'Target': pred_mapped})

# Check the submission DataFrame
print(submission.head())
print(f"Submission data shape: {submission.shape}")

# Save submission
submission.to_csv('submission.csv', index=False)


   id  Target
0   1       0
1   2       1
2   3       0
3   4       1
4   5       0
Submission data shape: (5, 2)


In [26]:
submission.to_csv('submission.csv', index=False)